### 골빈해커의 3분 딥러닝
#### 7장 이미지 인식의 은총알 CNN

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1]) # none은 입력데이터 개수, 1은 특징 개수(MNIST 회색조 이미지 색상이 1개)
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [4]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_normal([3, 3, 1 ,32], stddev=0.01))
    # Convolution layer
    L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding ='SAME')
    L1 = tf.nn.relu(L1)
    # Pooling layer
    L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
    # Convolution layer
    L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding ='SAME')
    L2 = tf.nn.relu(L2)
    # Pooling layer
    L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')

# full connect layer
with tf.name_scope('layer3'):
    W3 = tf.Variable(tf.random_normal([7 * 7 * 64,  256], stddev=0.01))
    L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
    L3 = tf.matmul(L3, W3)
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.dropout(L3, keep_prob)

with tf.name_scope('layer4'):
    W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
    model = tf.matmul(L3, W4)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = model))

    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    # optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [11]:
# 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict = {X: batch_xs, 
                                           Y: batch_ys, 
                                           keep_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: ', '%04d' % (epoch + 1),
          'Avg.cost =', '{:3f}'.format(total_cost / total_batch))

print('최적화 완료')

Epoch:  0001 Avg.cost = 0.174616
Epoch:  0002 Avg.cost = 0.048656
Epoch:  0003 Avg.cost = 0.031491
Epoch:  0004 Avg.cost = 0.021582
Epoch:  0005 Avg.cost = 0.014585
Epoch:  0006 Avg.cost = 0.013144
Epoch:  0007 Avg.cost = 0.012276
Epoch:  0008 Avg.cost = 0.008046
Epoch:  0009 Avg.cost = 0.005832
Epoch:  0010 Avg.cost = 0.008423
Epoch:  0011 Avg.cost = 0.008022
Epoch:  0012 Avg.cost = 0.008169
Epoch:  0013 Avg.cost = 0.003562
Epoch:  0014 Avg.cost = 0.008719
Epoch:  0015 Avg.cost = 0.004546
Epoch:  0016 Avg.cost = 0.004123
Epoch:  0017 Avg.cost = 0.004133
Epoch:  0018 Avg.cost = 0.005535
Epoch:  0019 Avg.cost = 0.001845
Epoch:  0020 Avg.cost = 0.004396
Epoch:  0021 Avg.cost = 0.010480
Epoch:  0022 Avg.cost = 0.003005
Epoch:  0023 Avg.cost = 0.001091
Epoch:  0024 Avg.cost = 0.000945
Epoch:  0025 Avg.cost = 0.000039
Epoch:  0026 Avg.cost = 0.000021
Epoch:  0027 Avg.cost = 0.000008
Epoch:  0028 Avg.cost = 0.000006
Epoch:  0029 Avg.cost = 0.000004
Epoch:  0030 Avg.cost = 0.000004
최적화 완료


In [6]:
# 결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, 
                           feed_dict= {X: mnist.test.images.reshape(-1,28,28,1),  
                                       Y: mnist.test.labels,
                                       keep_prob: 1}))

ValueError: Cannot feed value of shape (10000, 784) for Tensor 'Placeholder:0', which has shape '(?, 28, 28, 1)'

### 고수준 API - 보다 간단히..

In [7]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1]) 
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

with tf.name_scope('layer1'):
    L1 = tf.layers.conv2d(X, 32, [3, 3])
    L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
    L1 = tf.layers.dropout(L1, 0.7, is_training)
    
with tf.name_scope('layer2'):
    L2 = tf.layers.conv2d(L1, 64, [3, 3])
    L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
    L2 = tf.layers.dropout(L2, 0.7, is_training)

# full connect layer
with tf.name_scope('layer3'):
    L3 = tf.contrib.layers.flatten(L2)
    L3 = tf.layers.dense(L3, 256, activation = tf.nn.relu)
    L3 = tf.layers.dropout(L3, 0.5, is_training)

with tf.name_scope('layer4'):
    model = tf.layers.dense(L3, 10, activation = None)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = model))

    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    # optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [8]:
# 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict = {X: batch_xs, 
                                           Y: batch_ys, 
                                           keep_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: ', '%04d' % (epoch + 1),
          'Avg.cost =', '{:3f}'.format(total_cost / total_batch))

print('최적화 완료')

Epoch:  0001 Avg.cost = 0.169410
Epoch:  0002 Avg.cost = 0.046445
Epoch:  0003 Avg.cost = 0.029192
Epoch:  0004 Avg.cost = 0.021008
Epoch:  0005 Avg.cost = 0.016231
Epoch:  0006 Avg.cost = 0.012353
Epoch:  0007 Avg.cost = 0.010746
Epoch:  0008 Avg.cost = 0.008955
Epoch:  0009 Avg.cost = 0.007463
Epoch:  0010 Avg.cost = 0.009652
Epoch:  0011 Avg.cost = 0.005366
Epoch:  0012 Avg.cost = 0.007582
Epoch:  0013 Avg.cost = 0.006401
Epoch:  0014 Avg.cost = 0.005902
Epoch:  0015 Avg.cost = 0.006783
Epoch:  0016 Avg.cost = 0.002845
Epoch:  0017 Avg.cost = 0.004810
Epoch:  0018 Avg.cost = 0.005859
Epoch:  0019 Avg.cost = 0.003137
Epoch:  0020 Avg.cost = 0.004733
Epoch:  0021 Avg.cost = 0.004951
Epoch:  0022 Avg.cost = 0.005195
Epoch:  0023 Avg.cost = 0.001568
Epoch:  0024 Avg.cost = 0.004258
Epoch:  0025 Avg.cost = 0.008619
Epoch:  0026 Avg.cost = 0.002838
Epoch:  0027 Avg.cost = 0.002919
Epoch:  0028 Avg.cost = 0.004039
Epoch:  0029 Avg.cost = 0.002150
Epoch:  0030 Avg.cost = 0.004656
최적화 완료


In [10]:
# 결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, 
                           feed_dict= {X: mnist.test.images.reshape(-1,28,28,1), 
                                       Y: mnist.test.labels,
                                       keep_prob: 1}))

정확도:  0.9899
